In [1]:
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

2022-11-11 11:21:17.919672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Download the kaggle dataset and put it in ./data/

[Dataset](https://www.kaggle.com/datasets/uraninjo/augmented-alzheimer-mri-dataset/download?datasetVersionNumber=1)

In [7]:
import zipfile
import os
import shutil

zip_location = './data/archive.zip'
extract_to = './data/'

with zipfile.ZipFile(zip_location, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


os.remove("./data/archive.zip")
shutil.rmtree('./data/OriginalDataset') 


In [8]:
MobileNetv3Large = tf.keras.applications.MobileNetV3Large(
	input_shape=(224, 224, 3),
	include_top=False, 
	weights='imagenet')

2022-11-11 11:37:31.743592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


12683000/12683000 [==============================] - 2s 0us/step


In [14]:
data = tf.keras.preprocessing.image_dataset_from_directory('./data/AugmentedAlzheimerDataset/')


Found 33984 files belonging to 4 classes.


In [19]:
import math
# Create the training/test split 80/20
train_ds = data.take(math.floor(0.8 * len(data)))
test_ds = data.skip(math.ceil(0.8 * len(data)))

# Print the sizes of the training/test sets
print("Training set size: ", len(train_ds))
print("Test set size: ", len(test_ds))

Training set size:  849
Test set size:  212
